In [1]:
# To support both python 2 and python 3
from __future__ import division, print_function, unicode_literals

# Common imports
import numpy as np
import os
import tensorflow as tf
import time

# to make this notebook's output stable across runs
def reset_graph(seed=42): 
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

reset_graph()

In [2]:
# load data: digits 5 to 9, but still label with 0 to 4, 
# because TensorFlow expects label's integers from 0 to n_classes-1.
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/")

X_train2_full = mnist.train.images[mnist.train.labels >= 5]
y_train2_full = mnist.train.labels[mnist.train.labels >= 5] - 5
X_valid2_full = mnist.validation.images[mnist.validation.labels >= 5]
y_valid2_full = mnist.validation.labels[mnist.validation.labels >= 5] - 5
X_test2 = mnist.test.images[mnist.test.labels >= 5]
y_test2 = mnist.test.labels[mnist.test.labels >= 5] - 5

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [3]:
# we want to keep only 100 instances per class in the training set 
# and let's keep only 30 instances per class in the validation set
# tesing set is already loaded above
def sample_n_instances_per_class(X, y, n=100):
    Xs, ys = [], []
    for label in np.unique(y):
        idx = (y == label)
        Xc = X[idx][:n]
        yc = y[idx][:n]
        Xs.append(Xc)
        ys.append(yc)
    return np.concatenate(Xs), np.concatenate(ys)

X_train2, y_train2 = sample_n_instances_per_class(X_train2_full, y_train2_full, n=100)
X_valid2, y_valid2 = sample_n_instances_per_class(X_valid2_full, y_valid2_full, n=30)

In [7]:
learning_rate = 0.001
batch_size = 16
train_size = X_train2.shape[0]

#saver = tf.train.Saver()

with tf.Session() as sess:
    new_saver = tf.train.import_meta_graph('/Team48_HW2.ckpt.meta', clear_devices=True)
    new_saver.restore(sess, '/Team48_HW2.ckpt')
    xs = tf.get_default_graph().get_tensor_by_name("xs:0")
    ys = tf.get_default_graph().get_tensor_by_name("ys:0")
    y = tf.get_default_graph().get_tensor_by_name("y:0")
    #logits = y.op.inputs[0]
    accuracy = tf.get_default_graph().get_tensor_by_name("accuracy:0")
    output_layer_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope="logits")
    loss = tf.get_default_graph().get_tensor_by_name("loss:0")
    optimizer = tf.train.AdamOptimizer(learning_rate, name="optimizer")
    training_op = optimizer.minimize(loss, var_list=output_layer_vars, name="training_op")
    
    tf.global_variables_initializer().run() 
    
    best_loss = 99999
    stand_epoch = 0
    
    t0 = time.time()
    
    for i in range(1000):
        #sess.run(training_op, feed_dict={xs: X_train2, ys: y_train2})
        stand_epoch = stand_epoch + 1
        for j in range(int(train_size/batch_size)):
            # get batch data each time
            batch_x = X_train2[j*batch_size:(j+1)*batch_size,:]
            batch_y = y_train2[j*batch_size:(j+1)*batch_size]
            # training
            sess.run(training_op, feed_dict={xs: batch_x, ys: batch_y})
        
        val_loss = sess.run(loss, feed_dict={xs: X_valid2, ys: y_valid2})
        if best_loss > val_loss:
            stand_epoch = 0
            best_loss = val_loss
            
        
        accu = sess.run(accuracy, feed_dict={xs: X_test2, ys: y_test2})
        print(i,"Validation loss:", val_loss, " ; Best loss:", best_loss, " ; Accuracy:", accu)
        
        if stand_epoch >= 20: # Early stop
            print("Early stop in epoch", i)
            break
    t1 = time.time()
    save_path = new_saver.save(sess, "/Team48_HW3_1.ckpt") # save variables to checkpoint

sess.close()
print("Training time:", round(t1-t0,2), "sec")
print("Training Finish!!!")

INFO:tensorflow:Restoring parameters from /Team48_HW2.ckpt
0 Validation loss: 233.947  ; Best loss: 233.947  ; Accuracy: 0.350957
1 Validation loss: 221.941  ; Best loss: 221.941  ; Accuracy: 0.439004
2 Validation loss: 206.913  ; Best loss: 206.913  ; Accuracy: 0.561613
3 Validation loss: 197.253  ; Best loss: 197.253  ; Accuracy: 0.628266
4 Validation loss: 191.99  ; Best loss: 191.99  ; Accuracy: 0.660564
5 Validation loss: 187.955  ; Best loss: 187.955  ; Accuracy: 0.683604
6 Validation loss: 184.843  ; Best loss: 184.843  ; Accuracy: 0.699856
7 Validation loss: 182.41  ; Best loss: 182.41  ; Accuracy: 0.712816
8 Validation loss: 180.491  ; Best loss: 180.491  ; Accuracy: 0.725777
9 Validation loss: 178.959  ; Best loss: 178.959  ; Accuracy: 0.737503
10 Validation loss: 177.711  ; Best loss: 177.711  ; Accuracy: 0.743674
11 Validation loss: 176.668  ; Best loss: 176.668  ; Accuracy: 0.751286
12 Validation loss: 175.776  ; Best loss: 175.776  ; Accuracy: 0.756017
13 Validation loss:

In [8]:
with tf.Session() as sess:
    new_saver = tf.train.import_meta_graph('/Team48_HW2.ckpt.meta', clear_devices=True)
    new_saver.restore(sess, '/Team48_HW2.ckpt')
    xs = tf.get_default_graph().get_tensor_by_name("xs:0")
    ys = tf.get_default_graph().get_tensor_by_name("ys:0")
    y = tf.get_default_graph().get_tensor_by_name("y:0")
    accuracy = tf.get_default_graph().get_tensor_by_name("accuracy:0")
    
    layer5 = tf.get_default_graph().get_tensor_by_name("l5:0")
    output_layer_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope="logits")
    loss = tf.get_default_graph().get_tensor_by_name("loss:0")
    optimizer = tf.train.AdamOptimizer(learning_rate, name="optimizer")
    training_op = optimizer.minimize(loss, var_list=output_layer_vars, name="training_op")
    
    tf.global_variables_initializer().run()
    X_train_val = np.concatenate((X_train2, X_valid2))
    layer5_cache = sess.run(layer5, feed_dict={xs: X_train_val, ys: y_train2})
    
    best_loss = 99999
    stand_epoch = 0
    
    t3 = time.time()
    for i in range(1000):
        
        stand_epoch = stand_epoch + 1
        for j in range(int(train_size/batch_size)):
            # get batch data each time
            batch_x = layer5_cache[j*batch_size:(j+1)*batch_size,:]
            batch_y = y_train2[j*batch_size:(j+1)*batch_size]
            # training
            sess.run(training_op, feed_dict={layer5: batch_x, ys: batch_y})
        
        val_loss = sess.run(loss, feed_dict={layer5: layer5_cache[500:,:], ys: y_valid2})
        if best_loss > val_loss:
            stand_epoch = 0
            best_loss = val_loss
            
        accu = sess.run(accuracy, feed_dict={xs: X_test2, ys: y_test2})
        print(i,"Validation loss:", val_loss, " ; Best loss:", best_loss, " ; Accuracy:", accu)
        
        if stand_epoch >= 20: # Early stop
            print("Early stop in epoch", i)
            break
    
    t4 = time.time()
    save_path = new_saver.save(sess, "/Team48_HW3_2.ckpt") # save variables to checkpoint

print(round(t4-t3,2))
print("Training Finish!!!")
sess.close()

INFO:tensorflow:Restoring parameters from /Team48_HW2.ckpt
0 Validation loss: 233.207  ; Best loss: 233.207  ; Accuracy: 0.360831
1 Validation loss: 220.533  ; Best loss: 220.533  ; Accuracy: 0.452993
2 Validation loss: 204.447  ; Best loss: 204.447  ; Accuracy: 0.578688
3 Validation loss: 194.577  ; Best loss: 194.577  ; Accuracy: 0.649661
4 Validation loss: 189.304  ; Best loss: 189.304  ; Accuracy: 0.672084
5 Validation loss: 185.459  ; Best loss: 185.459  ; Accuracy: 0.696153
6 Validation loss: 182.587  ; Best loss: 182.587  ; Accuracy: 0.715079
7 Validation loss: 180.404  ; Best loss: 180.404  ; Accuracy: 0.726805
8 Validation loss: 178.709  ; Best loss: 178.709  ; Accuracy: 0.736474
9 Validation loss: 177.356  ; Best loss: 177.356  ; Accuracy: 0.745114
10 Validation loss: 176.243  ; Best loss: 176.243  ; Accuracy: 0.751491
11 Validation loss: 175.306  ; Best loss: 175.306  ; Accuracy: 0.755812
12 Validation loss: 174.505  ; Best loss: 174.505  ; Accuracy: 0.760132
13 Validation l